In [13]:
import pandas as pd
import numpy as np
import datetime
import os
%matplotlib inline

/usr/lib64/python3.4/site-packages/matplotlib/backends/backend_gtk3.py:215: Warning: Source ID 7 was not found when attempting to remove it
  GLib.source_remove(self._idle_event_id)


In [2]:
from kaqt.backtest.data import quotes_data, trades_data

In [3]:
symbol = 'EDZ9'

In [5]:
quotes = quotes_data(symbol=symbol)

In [8]:
quotes = quotes.ffill()

In [9]:
quotes.tail()

,Bid,BidSize,Ask,AskSize
DateTime,,,,
2015-06-23 23:58:44.320134,97.04,1,97.045,286
2015-06-23 23:58:44.352359,97.04,1,97.045,287
2015-06-23 23:59:15.160400,97.04,1,97.045,293
2015-06-23 23:59:15.664259,97.04,1,97.045,287
2015-06-23 23:59:17.792140,97.04,1,97.045,292


In [10]:
from kaqt.backtest.data import clean_quotes

In [11]:
quotes = clean_quotes(quotes)

In [16]:
quotes = quotes.reset_index()

In [22]:
quotes.DateTime = quotes.DateTime.astype(np.int64)

In [23]:
quotes.tail()

,DateTime,Bid,BidSize,Ask,AskSize
588130,1435103924320134000,97.04,1,97.045,286
588131,1435103924352359000,97.04,1,97.045,287
588132,1435103955160400000,97.04,1,97.045,293
588133,1435103955664259000,97.04,1,97.045,287
588134,1435103957792140000,97.04,1,97.045,292


In [15]:
from tables import *

In [24]:
class Quote(IsDescription):
    date_time = Int64Col()
    bid = Float64Col()
    ask = Float64Col()
    bid_size = Int32Col()
    ask_size = Int32Col()

In [27]:
filters = Filters(complib='zlib', complevel=9)

In [28]:
store = open_file('quotes.h5', 'w', filters=filters)

In [29]:
root = store.root

In [30]:
table = store.create_group(root, "EDZ9")

In [ ]:
table = store.get_node('')

In [31]:
tb = store.create_table(table, "20150623", Quote)

In [32]:
row = tb.row

In [33]:
for i in quotes.index:
    row['date_time'] = quotes.ix[i, 'DateTime']
    row['bid'] = quotes.ix[i, 'Bid']
    row['ask'] = quotes.ix[i, 'Ask']
    row['bid_size'] = quotes.ix[i, 'BidSize']
    row['ask_size'] = quotes.ix[i, 'AskSize']
    row.append()

tb.flush()

In [34]:
store.close()

In [35]:
store = pd.HDFStore('quotes.h5')

In [36]:
edz9 = store['EDZ9/D20150623']

In [37]:
edz9.tail()

,ask,ask_size,bid,bid_size,date_time
588130,97.045,286,97.04,1,1435103924320134000
588131,97.045,287,97.04,1,1435103924352359000
588132,97.045,293,97.04,1,1435103955160400000
588133,97.045,287,97.04,1,1435103955664259000
588134,97.045,292,97.04,1,1435103957792140000


In [39]:
edz9.index = pd.to_datetime(edz9.date_time)

In [40]:
edz9.tail()

,ask,ask_size,bid,bid_size,date_time
date_time,,,,,
2015-06-23 23:58:44.320134,97.045,286,97.04,1,1435103924320134000
2015-06-23 23:58:44.352359,97.045,287,97.04,1,1435103924352359000
2015-06-23 23:59:15.160400,97.045,293,97.04,1,1435103955160400000
2015-06-23 23:59:15.664259,97.045,287,97.04,1,1435103955664259000
2015-06-23 23:59:17.792140,97.045,292,97.04,1,1435103957792140000


In [41]:
store.close()